[Documentation of LBMPy](https://mybinder.org/v2/gh/mabau/lbmpy/master?filepath=doc%2Fnotebooks)

[Enhanced MRT](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.102.023307)

[MRT Theory](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.61.6546)

[Ulf repo](https://github.com/uschille/amrex-tutorials/blob/binary-fluctuating/GuidedTutorials/LBM/binary/binary-mrt-d3q19.ipynb)

1. [Analysis](#Moment-space-for-Ternary-Free-Energy-LBM)
    1. [Fluid 1](#Fluid-1)
    2. [Fluid 2](#Fluid-2)
    3. [Fluid 3](#Fluid-2)
2. [Gross et al. replication](#D2Q9-fluctuation-replication)

[Testing](#Test-Bed)

In [2]:
from lbmpy.session import *
import numpy as np
from lbmpy.moments import MOMENT_SYMBOLS, moment_matrix

# Moment space for Ternary Free Energy LBM

In [2]:
# import sympy.abc as symbolic
# from lbmpy.chapman_enskog import ChapmanEnskogAnalysis
# from lbmpy.methods.creationfunctions import create_from_equilibrium
# from lbmpy.methods import mrt_orthogonal_modes_literature
# from lbmpy.stencils import get_stencil
# from lbmpy.methods import DensityVelocityComputation
# from lbmpy.moments import moment_matrix, moments_up_to_component_order, exponents_to_polynomial_representations

In [3]:
d3q19 = LBStencil(Stencil.D3Q19)
v = sp.Matrix(d3q19)

In [4]:
five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

w_i = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

In [5]:
x, y, z = MOMENT_SYMBOLS
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
phi = sp.symbols('phi',cls=sp.Function)(x,y,z)
psi = sp.symbols('psi',cls=sp.Function)(x,y,z)
gamma_p = sp.symbols("\Gamma_{\phi}")
gamma_s = sp.symbols('\Gamma_{\psi}')
mu_p = sp.symbols("\mu_{\phi}")
mu_s = sp.symbols("\mu_{\psi}")

u = sp.Matrix(sp.symbols('u_0, u_1, u_2'))
cs = sp.sqrt(sp.Rational(1,3))
pb = sp.symbols('p_b')
deviation = False

krr = sp.Symbol(r"k_{\rho\rho}")
kpp = sp.Symbol(r"k_{\phi\phi}")
kss = sp.Symbol(r"k_{\psi\psi}")

one = sp.core.sympify(1)
two = sp.core.sympify(2)
three = sp.core.sympify(3)
five = sp.core.sympify(5)
six = sp.core.sympify(6)

c2 = x**2 + y**2 + z**2
c4 = c2**2

zeroth_order = [one]
first_order = [x, y, z]
second_order = [c2 - 1, 3*x**2 - c2, y**2 - z**2, x*y, y*z, z*x]
third_order = [(3*c2 - 5)*x, (3*c2 - 5)*y, (3*c2 - 5)*z, (y**2 - z**2)*x, (z**2 - x**2)*y, (x**2 - y**2)*z]
fourth_order = [3*c4 - 6*c2 + 1, (2*c2 - 3)*(3*x**2 - c2), (2*c2 - 3)*(y**2 - z**2)]

moments = [*zeroth_order, *first_order, *second_order, *third_order, *fourth_order]

In [6]:
M = moment_matrix(moments, stencil=d3q19)
M

⎡1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ⎤
⎢                                                                          ⎥
⎢0   0   0   -1  1   0   0   -1  1   -1  1   0   0   -1  1   0   0   -1  1 ⎥
⎢                                                                          ⎥
⎢0   1   -1  0   0   0   0   1   1   -1  -1  1   -1  0   0   1   -1  0   0 ⎥
⎢                                                                          ⎥
⎢0   0   0   0   0   1   -1  0   0   0   0   1   1   1   1   -1  -1  -1  -1⎥
⎢                                                                          ⎥
⎢-1  0   0   0   0   0   0   1   1   1   1   1   1   1   1   1   1   1   1 ⎥
⎢                                                                          ⎥
⎢0   -1  -1  2   2   -1  -1  1   1   1   1   -2  -2  1   1   -2  -2  1   1 ⎥
⎢                                                                          ⎥
⎢0   1   1   0   0   -1  -1  1   1   1   1   0   0   -1  -1  0   0   -1  -1⎥

## Fluid 1

In [7]:
coords = [x, y, z]

# Interfacial energy terms
del_rho = sum([sp.Derivative(rho, co, 2) for co in coords ])
del_phi = sum([sp.Derivative(phi, co, 2) for co in coords ])
del_psi = sum([sp.Derivative(psi, co, 2) for co in coords ])

sum_energy = krr*rho*del_rho + kpp*phi*del_phi + kss*psi*del_psi

# mixing terms

mix_rho = sp.Matrix([[sp.Derivative(rho, co1)*sp.Derivative(rho, co2) for co2 in coords] for co1 in coords])
mix_phi = sp.Matrix([[sp.Derivative(phi, co1)*sp.Derivative(phi, co2) for co2 in coords] for co1 in coords])
mix_psi = sp.Matrix([[sp.Derivative(psi, co1)*sp.Derivative(psi, co2) for co2 in coords] for co1 in coords])

mix_terms = krr*mix_rho + kpp*mix_phi + kss*mix_psi

In [8]:
def f_i(i):
    # Term 1
    e = v[i, :]
    e2 = e.T*e
    u2 = u*u.T
    dab = sp.eye(3)
    
    term1 = w_i[i]*(pb/cs**2 + rho*sum(e*u)/cs**2 + (1/(2*cs**4)*rho*(u.T@(e2 - cs**2*dab))@u)[0])
    
    # Term 2
    term2 = -w_i[i]/cs**2*sum_energy
    
    # Term 3
    trace = sum([ ww[i][j][j]*mix_terms[j,j] for j in range(3) ])
    traceless = sum([ ww[i][j][(j+1)%3]*mix_terms[j, (j+1)%3] for j in range(3) ])
    term3 = (trace + traceless)/cs**2
    
    return term1 + term2 + term3

feq = sp.Matrix([f_i(i) for i in range(19)])
feq[0] = rho - sum(feq[1:])

In [9]:
mf = M@feq
mf = sp.simplify(mf)
mf

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                              2                                           2  
⎢                             ∂                                           ∂   
⎢- 3⋅k_{\phi\phi}⋅φ(x, y, z)⋅───(φ(x, y, z)) - 3⋅k_{

## Fluid 2

In [10]:
def g_i(i):
    # Term 1
    e = v[i, :]
    e2 = e.T*e
    u2 = u*u.T
    dab = sp.eye(3)
    
    term1 = w_i[i]*((gamma_p*mu_p)/cs**2 + phi*sum(e*u)/cs**2 + (1/(2*cs**4)*phi*(u.T@(e2 - cs**2*dab))@u)[0])
    return term1
    
geq = sp.Matrix([g_i(i) for i in range(19)])
geq[0] = phi - sum(geq[1:])

In [11]:
mg = M@geq
mg = sp.simplify(mg)
mg

⎡                                        φ(x, y, z)                           
⎢                                                                             
⎢                                      u₀⋅φ(x, y, z)                          
⎢                                                                             
⎢                                      u₁⋅φ(x, y, z)                          
⎢                                                                             
⎢                                      u₂⋅φ(x, y, z)                          
⎢                                                                             
⎢                               2                2                2           
⎢3⋅\Gamma_{\phi}⋅\mu_{\phi} + u₀ ⋅φ(x, y, z) + u₁ ⋅φ(x, y, z) + u₂ ⋅φ(x, y, z)
⎢                                                                             
⎢                              ⎛    2     2     2⎞                            
⎢                              ⎝2⋅u₀  - u₁  - u₂ ⎠⋅φ

## Fluid 3

In [12]:
def k_i(i):
    # Term 1
    e = v[i, :]
    e2 = e.T*e
    u2 = u*u.T
    dab = sp.eye(3)
    
    term1 = w_i[i]*((gamma_s*mu_s)/cs**2 + psi*sum(e*u)/cs**2 + (1/(2*cs**4)*psi*(u.T@(e2 - cs**2*dab))@u)[0])
    return term1
    
keq = sp.Matrix([k_i(i) for i in range(19)])
keq[0] = psi - sum(keq[1:])

In [13]:
mk = M@keq
mk = sp.simplify(mk)
mk

⎡                                        ψ(x, y, z)                           
⎢                                                                             
⎢                                      u₀⋅ψ(x, y, z)                          
⎢                                                                             
⎢                                      u₁⋅ψ(x, y, z)                          
⎢                                                                             
⎢                                      u₂⋅ψ(x, y, z)                          
⎢                                                                             
⎢                               2                2                2           
⎢3⋅\Gamma_{\psi}⋅\mu_{\psi} + u₀ ⋅ψ(x, y, z) + u₁ ⋅ψ(x, y, z) + u₂ ⋅ψ(x, y, z)
⎢                                                                             
⎢                              ⎛    2     2     2⎞                            
⎢                              ⎝2⋅u₀  - u₁  - u₂ ⎠⋅ψ

# D2Q9 fluctuation replication

In [8]:
d2q9 = LBStencil(Stencil.D2Q9)
v = sp.Matrix(d2q9)

In [10]:
x, y, z = MOMENT_SYMBOLS
zero = sp.core.sympify(0)
one = sp.core.sympify(1)

c2 = x**2 + y**2
c4 = c2**2

zeroth_order = [one]
first_order = [x, y]
second_order = [3*c2 - 2, 2*x**2 - c2, x*y]
third_order = [(3*c2 - 4)*x, (3*c2 - 4)*y, 9*c4 - 15*c2 + 2]

moments = [*zeroth_order, *first_order, *second_order, *third_order]
M = moment_matrix(moments, stencil=d2q9)
M

⎡1   1   1   1   1   1   1  1   1 ⎤
⎢                                 ⎥
⎢0   0   0   -1  1   -1  1  -1  1 ⎥
⎢                                 ⎥
⎢0   1   -1  0   0   1   1  -1  -1⎥
⎢                                 ⎥
⎢-2  1   1   1   1   4   4  4   4 ⎥
⎢                                 ⎥
⎢0   -1  -1  1   1   0   0  0   0 ⎥
⎢                                 ⎥
⎢0   0   0   0   0   -1  1  1   -1⎥
⎢                                 ⎥
⎢0   0   0   1   -1  -2  2  -2  2 ⎥
⎢                                 ⎥
⎢0   -1  1   0   0   2   2  -2  -2⎥
⎢                                 ⎥
⎣2   -4  -4  -4  -4  8   8  8   8 ⎦

In [15]:
four9 = sp.Rational(4, 9)
one12 = sp.Rational(1, 12)
one18 = sp.Rational(1, 18)
one36 = sp.Rational(1, 36)
one72 = sp.Rational(1, 72)


w_i = [four9] + [one18]*4 + [one36]*4

ww =  [ [[0]*2]*2 ] \
    + [ [ [  -one18, 0 ], [ 0, 2*one18 ] ] ]*2 \
    + [ [ [ 2*one18, 0 ], [ 0,  -one18 ] ] ]*2 \
    + [ [ [ -one72, -one12 ], [ -one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72, -one12 ], [ -one12, -one72 ] ] ]

# Test Bed

In [14]:
# equilibrium = lbmpy.equilibrium.GenericDiscreteEquilibrium(d3q19, feq, rho, u)

In [15]:
# omega = sp.symbols("omega")
# relaxation_rate_dict = {moment : omega for moment in moments}

# cqc = DensityVelocityComputation(d3q19, True, False)

# method = create_from_equilibrium(d3q19, equilibrium, cqc, relaxation_rate_dict)
# method